In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("C:/Users/Priyanshi Saxena/OneDrive/Desktop/Healthcare_Folder/healthcare-claims-funnel/data/raw/Final_Healthcare_dataset.csv")

In [3]:
df.head()

,claim_id,SSN,claim_age,patient_name,patient_id,provider_id,provider_name,claim_amount,approved_amount,claim_status,submission_date,adjudication_date,payment_date,denial_reason
0,CLM500000,806-63-5608,79,David Wilson,P00001,PR014,Anthem,15656,0,Denied,19-09-2024,28-09-2024,NaN,Timely Filing Limit Exceeded
1,CLM500001,341-10-1231,65,Susan Taylor,P00002,PR015,Molina Healthcare,39862,27932,Approved,28-04-2024,09-05-2024,07-06-2024,NaN
2,CLM500002,856-62-8239,22,Michael Wilson,P00003,PR006,Blue Cross Blue Shield,36899,25479,Approved,07-07-2024,11-07-2024,22-07-2024,NaN
3,CLM500003,227-12-8424,38,Isabella Williams,P00004,PR009,Vision Service Plan,49240,40359,Approved,05-04-2024,10-04-2024,02-05-2024,NaN
4,CLM500004,176-35-4417,27,Mark Davis,P00005,PR002,Medicaid,38208,35894,Approved,12-04-2024,26-04-2024,07-05-2024,NaN


In [4]:
df.shape

(12000, 14)

In [5]:
df.columns

Index(['claim_id', 'SSN', 'claim_age', 'patient_name', 'patient_id',
       'provider_id', 'provider_name', 'claim_amount', 'approved_amount',
       'claim_status', 'submission_date', 'adjudication_date', 'payment_date',
       'denial_reason'],
      dtype='object')

In [6]:
df.info

<bound method DataFrame.info of         claim_id          SSN  claim_age       patient_name patient_id  \
0      CLM500000  806-63-5608         79       David Wilson     P00001   
1      CLM500001  341-10-1231         65       Susan Taylor     P00002   
2      CLM500002  856-62-8239         22     Michael Wilson     P00003   
3      CLM500003  227-12-8424         38  Isabella Williams     P00004   
4      CLM500004  176-35-4417         27         Mark Davis     P00005   
...          ...          ...        ...                ...        ...   
11995  CLM511995  303-86-6459         44      Matthew Brown     P00049   
11996  CLM511996  909-19-7215         56      Daniel Wilson     P00255   
11997  CLM511997  196-31-2439         56     Rosie Martinez     P00299   
11998  CLM511998  828-30-8885         84     Mark Rodriguez     P00147   
11999  CLM511999  780-65-5992         24    Thomas Anderson     P00323   

      provider_id           provider_name  claim_amount  approved_amount  \
0  

In [7]:
df.isnull().sum()

claim_id                0
SSN                     0
claim_age               0
patient_name            0
patient_id              0
provider_id             0
provider_name           0
claim_amount            0
approved_amount         0
claim_status            0
submission_date         0
adjudication_date       0
payment_date         3135
denial_reason        8865
dtype: int64

In [8]:
df.duplicated().sum()

np.int64(0)

In [9]:
df = df.drop_duplicates()


In [10]:
date_cols = ["submission_date", "adjudication_date", "payment_date"]

for col in date_cols:
    df[col] = pd.to_datetime(
        df[col],
        errors="coerce",
        dayfirst=True
    )

In [11]:
df.duplicated(subset=['claim_id']).sum()

np.int64(0)

In [12]:
df["claim_status"] = df["claim_status"].str.strip().str.title()
df["provider_name"] = df["provider_name"].str.strip()
df["denial_reason"] = df["denial_reason"].str.strip()

In [13]:
df["is_approved"] = np.where(df["claim_status"] == "Approved", 1, 0)


In [14]:
df["is_denied"] = np.where(df["claim_status"] == "Denied", 1, 0)


In [15]:
df["adjudication_days"] = (
    df["adjudication_date"] - df["submission_date"]
).dt.days


In [16]:
df["payment_delay_days"] = (
    df["payment_date"] - df["adjudication_date"]
).dt.days


In [17]:
# Approved claims must have payment_date
df[(df["claim_status"] == "Approved") & (df["payment_date"].isna())].shape


(0, 18)

In [18]:
# Denied claims must have denial_reason
df[(df["claim_status"] == "Denied") & (df["denial_reason"] == "")].shape

(0, 18)

In [19]:
df = df.drop(columns=["SSN"])

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   claim_id            12000 non-null  object        
 1   claim_age           12000 non-null  int64         
 2   patient_name        12000 non-null  object        
 3   patient_id          12000 non-null  object        
 4   provider_id         12000 non-null  object        
 5   provider_name       12000 non-null  object        
 6   claim_amount        12000 non-null  int64         
 7   approved_amount     12000 non-null  int64         
 8   claim_status        12000 non-null  object        
 9   submission_date     12000 non-null  datetime64[ns]
 10  adjudication_date   12000 non-null  datetime64[ns]
 11  payment_date        8865 non-null   datetime64[ns]
 12  denial_reason       3135 non-null   object        
 13  is_approved         12000 non-null  int64     

In [21]:
invalid_dates = df[(df["adjudication_date"] >= df["submission_date"])]
invalid_dates.shape

(12000, 17)

In [22]:
invalid_dates[['claim_id', 'submission_date', 'adjudication_date']].head(10)

,claim_id,submission_date,adjudication_date
0,CLM500000,2024-09-19,2024-09-28
1,CLM500001,2024-04-28,2024-05-09
2,CLM500002,2024-07-07,2024-07-11
3,CLM500003,2024-04-05,2024-04-10
4,CLM500004,2024-04-12,2024-04-26
5,CLM500005,2024-05-16,2024-05-20
6,CLM500006,2024-06-12,2024-06-19
7,CLM500007,2024-02-22,2024-03-02
8,CLM500008,2024-05-03,2024-05-20
9,CLM500009,2024-03-24,2024-03-30


In [23]:
df["submission_date"] = pd.to_datetime(
    df["submission_date"],
    dayfirst=True,
    errors="coerce"
)

df["adjudication_date"] = pd.to_datetime(
    df["adjudication_date"],
    dayfirst=True,
    errors="coerce"
)
df["payment_date"] = pd.to_datetime(
    df["payment_date"],
    dayfirst=True,
    errors="coerce"
)

In [24]:
invalid_dates = df[df["adjudication_date"] <= df["submission_date"]]
invalid_dates.shape

(0, 17)

In [25]:
df[["submission_date", "adjudication_date"]].head()

,submission_date,adjudication_date
0,2024-09-19,2024-09-28
1,2024-04-28,2024-05-09
2,2024-07-07,2024-07-11
3,2024-04-05,2024-04-10
4,2024-04-12,2024-04-26


In [26]:
df["approved_missing_payment_flag"] = (
    (df["claim_status"] == "Approved") & (df["payment_date"].isna())
).astype(int)
df[["approved_missing_payment_flag", "claim_status", "payment_date"]].head(10)

,approved_missing_payment_flag,claim_status,payment_date
0,0,Denied,NaT
1,0,Approved,2024-06-07
2,0,Approved,2024-07-22
3,0,Approved,2024-05-02
4,0,Approved,2024-05-07
5,0,Approved,2024-05-29
6,0,Approved,2024-07-01
7,0,Denied,NaT
8,0,Approved,2024-06-01
9,0,Approved,2024-04-20


In [27]:
df["denied_with_payment_flag"] = (
    (df["claim_status"] == "Denied") & (df["payment_date"].notna())
).astype(int)
df["denied_with_payment_flag"].sum()

np.int64(0)

In [28]:
patient_id_map = (df.groupby("patient_name")["patient_id"].first().to_dict())
df["patient_id"] = df["patient_name"].map(patient_id_map)

In [29]:
provider_id_map = (df.groupby("provider_name")["provider_id"].first().to_dict())
df["provider_id"] = df["provider_name"].map(provider_id_map)

In [30]:
df.groupby("patient_name")["patient_id"].nunique().sort_values(ascending=False).head()

patient_name
Andrew Anderson    1
Andrew Brown       1
Andrew Davis       1
Andrew Garcia      1
Andrew Gonzalez    1
Name: patient_id, dtype: int64

In [31]:
df["claim_funnel_stage"] = df["claim_status"].map({
    "Submitted": "Submitted",
    "In Review": "Adjudicated",
    "Approved": "Approved",
    "Denied": "Denied"
})

In [32]:
final_cols = [
    "claim_id",
    "claim_age",
    "patient_name",
    "patient_id",
    "provider_id",
    "provider_name",
    "claim_amount",
    "approved_amount",
    "claim_status",
    "submission_date",
    "adjudication_date",
    "payment_date",
    "denial_reason",
    "is_approved",
    "is_denied",
    "adjudication_days",
    "payment_delay_days",
    "approved_missing_payment_flag",
    "denied_with_payment_flag",
    "claim_funnel_stage"
]
df_final = df[final_cols]

In [33]:
df_final.info()
df_final.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   claim_id                       12000 non-null  object        
 1   claim_age                      12000 non-null  int64         
 2   patient_name                   12000 non-null  object        
 3   patient_id                     12000 non-null  object        
 4   provider_id                    12000 non-null  object        
 5   provider_name                  12000 non-null  object        
 6   claim_amount                   12000 non-null  int64         
 7   approved_amount                12000 non-null  int64         
 8   claim_status                   12000 non-null  object        
 9   submission_date                12000 non-null  datetime64[ns]
 10  adjudication_date              12000 non-null  datetime64[ns]
 11  payment_date   

,claim_id,claim_age,patient_name,patient_id,provider_id,provider_name,claim_amount,approved_amount,claim_status,submission_date,adjudication_date,payment_date,denial_reason,is_approved,is_denied,adjudication_days,payment_delay_days,approved_missing_payment_flag,denied_with_payment_flag,claim_funnel_stage
0,CLM500000,79,David Wilson,P00001,PR014,Anthem,15656,0,Denied,2024-09-19,2024-09-28,NaT,Timely Filing Limit Exceeded,0,1,9,NaN,0,0,Denied
1,CLM500001,65,Susan Taylor,P00002,PR015,Molina Healthcare,39862,27932,Approved,2024-04-28,2024-05-09,2024-06-07,NaN,1,0,11,29.0,0,0,Approved
2,CLM500002,22,Michael Wilson,P00003,PR006,Blue Cross Blue Shield,36899,25479,Approved,2024-07-07,2024-07-11,2024-07-22,NaN,1,0,4,11.0,0,0,Approved
3,CLM500003,38,Isabella Williams,P00004,PR009,Vision Service Plan,49240,40359,Approved,2024-04-05,2024-04-10,2024-05-02,NaN,1,0,5,22.0,0,0,Approved
4,CLM500004,27,Mark Davis,P00005,PR002,Medicaid,38208,35894,Approved,2024-04-12,2024-04-26,2024-05-07,NaN,1,0,14,11.0,0,0,Approved


In [34]:
df.to_csv("C:/Users/Priyanshi Saxena/OneDrive/Desktop/Healthcare_Folder/healthcare-claims-funnel/data/cleaned/healthcare_claims_cleaned_final.csv", index=False)